In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import itertools

In [2]:
from google.colab import files
uploaded = files.upload()

Saving penn-data.json to penn-data.json


Reading the Data

In [162]:
# Reading data from Json file and storing it to dataframe
raw_data = pd.read_json("penn-data.json")

Initialising the Emmision matrix with all 1 in row, columns

In [163]:
sz = len(raw_data)
df = pd.DataFrame()
tag_li = []
word_li = []

for i in range(sz):
  rd = raw_data.iloc[i]
  st = str(rd[0])
  lw =re.sub(r'[^\w\s]', '', st).lower()
  lw = lw.split(" ")
  raw_data.iloc[i][0]= lw
  
  tag_li.append(rd[1])
  word_li.append(lw)
tags = []
words = []
for lst in tag_li:
  for tag in lst:
    tags.append(tag)
tagss = set(tags)
for wrd in word_li:
  for word in wrd:
    words.append(word)
wordss = set(words)
print(len(words)) 
df = pd.DataFrame(index = tagss, columns = wordss)
df= df.fillna(1)


81147


Dividing the dataset into train test and test set in 80:20 Ration

In [164]:
train_set,test_set =train_test_split(raw_data,train_size=0.80,test_size=0.20,random_state = 101)
print(train_set)

                                                      0                                                  1
1998          [drink, carrier, competes, with, cartons]                             [NN, NN, VBZ, IN, NNS]
1455  [at, last, count, candela, had, sold, 4, milli...  [IN, JJ, NN, NNP, VBD, VBN, CD, CD, IN, PRP$, ...
218   [mrs, hills, lauded, south, korea, for, creati...  [NNP, NNP, VBD, NNP, NNP, IN, VBG, DT, JJ, NN,...
2161  [section, 501, of, the, united, states, inform...  [NN, CD, IN, DT, NNP, NNPS, NNP, CC, NNP, NNP,...
7     [a, lorillard, spokewoman, said, this, is, an,...            [DT, NNP, NN, VBD, DT, VBZ, DT, JJ, NN]
...                                                 ...                                                ...
1599  [the, programtrading, issue, is, heating, up, ...  [DT, JJ, NN, VBZ, VBG, RP, IN, NNP, NNP, IN, P...
3910  [sen, kennedy, said, in, a, separate, statemen...  [NNP, NNP, VBD, IN, DT, JJ, NN, IN, PRP, VBZ, ...
1361  [economic, news, had, little, e

In [165]:
# creating List of all words and tags
tagL = []
wordL = []
nn = len(train_set)
for i in range(nn):
  rd = train_set.iloc[i]
  st = str(rd[0])
  lw =re.sub(r'[^\w\s]', '', st).lower()
  lw = lw.split(" ")
  train_set.iloc[i][0]= lw
  
  tagL.append(rd[1])
  wordL.append(lw)
ln = len(tagL)


In [166]:
# Storing frequency of all word for given tag in emmision Matrix
for i in range(len(train_set)):
  rd = train_set.iloc[i]
  n = len(rd[0])
  
  tg = rd[1]
  wd = rd[0]

  for j in range(n):
    df.loc[tg[j],wd[j]]+=1
df


,,openly,tanked,cigna,patent,polystyrene,microprocessor,workable,published,file,lines,mossman,occurs,645000,ties,helps,reddington,needy,hotel,01,original,witness,13101,increasing,newspaper,152000,enraged,alleviate,chunks,riese,criticism,hammerton,bankroll,hero,flourish,stupid,attention,accounts,treasury,discontinue,...,eaton,after,truthful,ticket,facilitate,grange,1980,believed,understatement,incorporated,reporters,impending,ought,386,available,drink,98,minn,ten,released,droughts,rapport,waste,familyplanning,pressman,strapped,industry,collective,inaccurately,chassis,sewer,extensive,guffey,313,receivers,subcommittees,switched,passenger,routine,vision
WRB,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
FW,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
DT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
UH,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
RBR,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
NNP,1,1,1,3,1,1,1,1,1,1,2,3,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,4,2,1,1,1,1,1,1,1,28,1,...,2,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,4,1,1,1,1,1,3,1,1,1,1,1,1,1
PDT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PRP,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
NN,1,1,1,1,8,2,2,1,1,1,1,1,1,1,1,1,1,1,3,1,1,2,1,1,9,1,1,1,1,1,2,1,1,3,1,1,6,1,1,1,...,1,1,1,2,1,1,1,1,2,1,1,1,1,1,1,3,1,1,1,1,1,2,2,1,2,1,31,1,1,2,2,1,1,1,1,2,1,3,2,2
VBD,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,4,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1


Calculating prior probability of each tag

In [167]:
p_p=[0]*(len(tagss))
tagL = len(tags)
for i in tagss:
  count=0
  for j in tags:
    if(i==j):
      count+=1
    ind  = list(tagss).index(i)
    p_p[ind] = count/tagL
p_p



[0.0021935499772018684,
 4.929325791464872e-05,
 0.1006198627182767,
 3.6969943435986544e-05,
 0.0016759707690980567,
 0.11596238924421112,
 0.0003327294909238789,
 0.0211468076453843,
 0.1567648834830616,
 0.03749984595856902,
 0.005483874943004671,
 0.02791230729416984,
 0.026852502249004893,
 0.026297953097465094,
 1.232331447866218e-05,
 0.0001232331447866218,
 0.00943965889065523,
 0.03147374517850321,
 1.232331447866218e-05,
 0.0745190826524702,
 0.0014787977374394617,
 0.12147091081617312,
 0.00017252640270127054,
 0.030771316253219466,
 0.0015527376243114348,
 0.0030808286196655453,
 0.011362095949326531,
 0.003006888732793572,
 0.00019717303165859489,
 0.017992039138846783,
 0.015761519218208928,
 0.0029699187893575856,
 0.0436984731413361,
 0.002661835927391031,
 0.0010844516741222718,
 0.004695182816370291,
 0.02491774187585493,
 0.0718818933540365,
 0.002242843235116517,
 0.00016020308822260835,
 0.0004313160067531763]

Calculation of emmision Probability and storing it to emmision table

In [168]:
nt = len(df.iloc[0])
for i in range(nt):
  s = df.iloc[:,i].sum()
  df.iloc[:,i]/=s
df


,,openly,tanked,cigna,patent,polystyrene,microprocessor,workable,published,file,lines,mossman,occurs,645000,ties,helps,reddington,needy,hotel,01,original,witness,13101,increasing,newspaper,152000,enraged,alleviate,chunks,riese,criticism,hammerton,bankroll,hero,flourish,stupid,attention,accounts,treasury,discontinue,...,eaton,after,truthful,ticket,facilitate,grange,1980,believed,understatement,incorporated,reporters,impending,ought,386,available,drink,98,minn,ten,released,droughts,rapport,waste,familyplanning,pressman,strapped,industry,collective,inaccurately,chassis,sewer,extensive,guffey,313,receivers,subcommittees,switched,passenger,routine,vision
WRB,0.003012,0.023256,0.023810,0.023256,0.020833,0.023810,0.023810,0.023810,0.023810,0.021739,0.021739,0.023256,0.023256,0.023256,0.021739,0.023810,0.023810,0.023810,0.022222,0.022727,0.022222,0.023810,0.023810,0.018519,0.020408,0.023810,0.02439,0.023810,0.02439,0.022727,0.023256,0.02439,0.023810,0.023256,0.023810,0.022222,0.021739,0.020833,0.014706,0.023810,...,0.023810,0.008850,0.023810,0.023810,0.023810,0.023810,0.023810,0.021277,0.023810,0.023810,0.022222,0.023810,0.023256,0.023256,0.018182,0.022222,0.022222,0.02439,0.02439,0.022222,0.023256,0.023810,0.022727,0.023256,0.023810,0.023810,0.013514,0.023810,0.023810,0.023810,0.023810,0.023810,0.023256,0.023810,0.02439,0.023810,0.023256,0.023256,0.023256,0.023810
FW,0.003012,0.023256,0.023810,0.023256,0.020833,0.023810,0.023810,0.023810,0.023810,0.021739,0.021739,0.023256,0.023256,0.023256,0.021739,0.023810,0.023810,0.023810,0.022222,0.022727,0.022222,0.023810,0.023810,0.018519,0.020408,0.023810,0.02439,0.023810,0.02439,0.022727,0.023256,0.02439,0.023810,0.023256,0.023810,0.022222,0.021739,0.020833,0.014706,0.023810,...,0.023810,0.008850,0.023810,0.023810,0.023810,0.023810,0.023810,0.021277,0.023810,0.023810,0.022222,0.023810,0.023256,0.023256,0.018182,0.022222,0.022222,0.02439,0.02439,0.022222,0.023256,0.023810,0.022727,0.023256,0.023810,0.023810,0.013514,0.023810,0.023810,0.023810,0.023810,0.023810,0.023256,0.023810,0.02439,0.023810,0.023256,0.023256,0.023256,0.023810
DT,0.003012,0.023256,0.023810,0.023256,0.020833,0.023810,0.023810,0.023810,0.023810,0.021739,0.021739,0.023256,0.023256,0.023256,0.021739,0.023810,0.023810,0.023810,0.022222,0.022727,0.022222,0.023810,0.023810,0.018519,0.020408,0.023810,0.02439,0.023810,0.02439,0.022727,0.023256,0.02439,0.023810,0.023256,0.023810,0.022222,0.021739,0.020833,0.014706,0.023810,...,0.023810,0.008850,0.023810,0.023810,0.023810,0.023810,0.023810,0.021277,0.023810,0.023810,0.022222,0.023810,0.023256,0.023256,0.018182,0.022222,0.022222,0.02439,0.02439,0.022222,0.023256,0.023810,0.022727,0.023256,0.023810,0.023810,0.013514,0.023810,0.023810,0.023810,0.023810,0.023810,0.023256,0.023810,0.02439,0.023810,0.023256,0.023256,0.023256,0.023810
UH,0.003012,0.023256,0.023810,0.023256,0.020833,0.023810,0.023810,0.023810,0.023810,0.021739,0.021739,0.023256,0.023256,0.023256,0.021739,0.023810,0.023810,0.023810,0.022222,0.022727,0.022222,0.023810,0.023810,0.018519,0.020408,0.023810,0.02439,0.023810,0.02439,0.022727,0.023256,0.02439,0.023810,0.023256,0.023810,0.022222,0.021739,0.020833,0.014706,0.023810,...,0.023810,0.008850,0.023810,0.023810,0.023810,0.023810,0.023810,0.021277,0.023810,0.023810,0.022222,0.023810,0.023256,0.023256,0.018182,0.022222,0.022222,0.02439,0.02439,0.022222,0.023256,0.023810,0.022727,0.023256,0.023810,0.023810,0.013514,0.023810,0.023810,0.023810,0.023810,0.023810,0.023256,0.023810,0.02439,0.023810,0.023256,0.023256,0.023256,0.023810
RBR,0.003012,0.023256,0.023810,0.023256,0.020833,0.023810,0.023810,0.023810,0.023810,0.021739,0.021739,0.023256,0.023256,0.023256,0.021739,0.023810,0.023810,0.023810,0.022222,0.022727,0.022222,0.023810,0.023810,0.018519,0.020408,0.023810,0.02439,0.023810,0.02439,0.022727,0.023256,0.02439,0.023810,0.023256,0.023810,0.022222,0.021739,0.020833,0.014706,0.023810,...,0.023810,0.008850,0.023810,0.023810,0.023810,0.023810,0.023810,0.021277,0.023810,0.023810,0.

Initialisng Trigram transition probability Dataframe with all 1

In [169]:
BitagL = []
nn = len(raw_data)
for i in tagss:
  for j in tagss:
    triT = (i,j)
    BitagL.append(triT)


dft = pd.DataFrame(index = BitagL,columns=tagss)
dft = dft.fillna(1)

dft



,WRB,FW,DT,UH,RBR,NNP,PDT,PRP,NN,VBD,WDT,CC,TO,VBN,SYM,'',PRP$,VB,",",NNS,-LRB-,IN,WP$,RB,-RRB-,:,MD,NNPS,#,VBG,VBP,WP,CD,RP,EX,JJR,VBZ,JJ,JJS,LS,RBS
"(WRB, WRB)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, FW)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, DT)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, UH)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, RBR)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(RBS, VBZ)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(RBS, JJ)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(RBS, JJS)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(RBS, LS)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [170]:
# storing frequency of all pair of tags suceeded by given tag
nn = len(train_set)
for i in range(nn):
  rd = train_set.iloc[i]
  tset = rd[1]
  for j in range(2,len(tset)):
    lt = (tset[j-2],tset[j-1])
    cur = tset[j]
    dft.loc[[lt],cur]+=1
dft


,WRB,FW,DT,UH,RBR,NNP,PDT,PRP,NN,VBD,WDT,CC,TO,VBN,SYM,'',PRP$,VB,",",NNS,-LRB-,IN,WP$,RB,-RRB-,:,MD,NNPS,#,VBG,VBP,WP,CD,RP,EX,JJR,VBZ,JJ,JJS,LS,RBS
"(WRB, WRB)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, FW)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, DT)",1,1,1,1,1,7,1,2,24,1,1,1,1,1,1,1,1,1,1,11,1,1,1,2,1,1,2,1,1,1,1,1,1,1,1,1,1,6,2,1,1
"(WRB, UH)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(WRB, RBR)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(RBS, VBZ)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(RBS, JJ)",1,1,1,1,1,3,1,2,10,1,1,1,1,1,1,1,1,1,1,5,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1
"(RBS, JJS)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(RBS, LS)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [171]:
# Setting probability value for trigram sequence in trigram transition probability matrix
nt = len(dft.iloc[0])
for i in range(nt):
  s = dft.iloc[:,i].sum()
  dft.iloc[:,i]/=s




Calculating Start Probability of each tag

In [172]:
tagInd = list(tagss)
s = len(tagss)
prior_prob = [1]*s
for i in range(nn):
  rd = train_set.iloc[i]
  ini_tag = rd[1][0]
  ind = tagInd.index(ini_tag)
  prior_prob[ind]+=1


In [173]:

sm = sum(prior_prob)
for i in range(len(prior_prob)):
  prior_prob[i] = prior_prob[i]/sm
print(prior_prob)

[0.006305170239596469, 0.00031525851197982345, 0.23865069356872637, 0.00031525851197982345, 0.0009457755359394704, 0.19546027742749053, 0.0012610340479192938, 0.09079445145018916, 0.04823455233291299, 0.0009457755359394704, 0.0009457755359394704, 0.054539722572509455, 0.004413619167717529, 0.0069356872635561164, 0.00031525851197982345, 0.00031525851197982345, 0.00914249684741488, 0.005359394703656999, 0.00031525851197982345, 0.04728877679697352, 0.0018915510718789407, 0.1358764186633039, 0.00031525851197982345, 0.05233291298865069, 0.00031525851197982345, 0.0028373266078184113, 0.00031525851197982345, 0.0031525851197982345, 0.00031525851197982345, 0.011664564943253467, 0.0012610340479192938, 0.005359394703656999, 0.01008827238335435, 0.00031525851197982345, 0.007881462799495587, 0.0037831021437578815, 0.0031525851197982345, 0.04129886506935687, 0.0018915510718789407, 0.0022068095838587644, 0.0009457755359394704]


Calculating Bigram Transition Probability Dataframe

In [174]:
# Initialising Bigram Dataframe
dfb = pd.DataFrame(index = tagss,columns =tagss)
dfb = dfb.fillna(1)

In [175]:
# Storing frequency of each pair of sequence of tag
nn = len(train_set)
for i in range(nn):
  rd = train_set.iloc[i]
  tset = rd[1]
  for j in range(1,len(tset)):
    dfb.loc[tset[j-1],tset[j]]+=1
dfb


,WRB,FW,DT,UH,RBR,NNP,PDT,PRP,NN,VBD,WDT,CC,TO,VBN,SYM,'',PRP$,VB,",",NNS,-LRB-,IN,WP$,RB,-RRB-,:,MD,NNPS,#,VBG,VBP,WP,CD,RP,EX,JJR,VBZ,JJ,JJS,LS,RBS
WRB,1,1,49,1,1,11,2,20,10,3,1,1,3,5,1,1,4,1,1,13,1,5,1,5,1,1,1,1,1,2,2,1,4,1,2,1,4,12,1,1,2
FW,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1
DT,2,2,14,1,11,835,1,8,3077,12,4,5,4,78,1,1,2,1,1,511,3,70,1,62,2,2,11,24,1,66,8,6,207,1,1,41,30,1351,57,1,17
UH,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
RBR,2,1,6,1,1,4,1,2,1,2,1,3,3,5,1,1,1,3,1,1,2,25,1,15,1,1,2,1,1,3,2,1,1,1,1,1,2,44,1,1,1
NNP,14,1,224,1,1,3158,1,28,650,556,53,354,56,41,1,1,3,11,2,223,33,399,4,96,34,20,84,130,1,33,44,37,228,2,1,6,317,177,9,1,3
PDT,1,1,21,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PRP,1,1,30,1,2,6,1,7,10,357,1,19,30,10,1,2,1,13,1,3,4,55,1,104,2,5,163,1,1,22,216,2,1,7,1,6,262,26,1,1,1
NN,33,3,256,1,10,258,1,130,1464,610,146,518,393,121,1,6,10,26,1,927,20,2605,3,236,26,66,182,2,1,145,48,32,59,5,10,24,553,181,9,1,2
VBD,7,1,420,2,9,160,2,154,98,13,1,11,146,265,1,1,58,31,1,61,1,306,1,140,1,1,5,1,1,63,2,1,165,38,5,22,4,137,2,1,1


In [176]:
# Using frequency to Calculate Probability of Bigram transition
nt = len(dfb.iloc[0])
for i in range(nt):
  s = dfb.iloc[:,i].sum()
  dfb.iloc[:,i]/=s
dfb

,WRB,FW,DT,UH,RBR,NNP,PDT,PRP,NN,VBD,WDT,CC,TO,VBN,SYM,'',PRP$,VB,",",NNS,-LRB-,IN,WP$,RB,-RRB-,:,MD,NNPS,#,VBG,VBP,WP,CD,RP,EX,JJR,VBZ,JJ,JJS,LS,RBS
WRB,0.006211,0.022727,0.008464,0.023256,0.006579,0.001613,0.032787,0.017652,0.000995,0.001215,0.002532,0.000591,0.001712,0.002900,0.02439,0.020408,0.006359,0.000483,0.023810,0.002735,0.007299,0.000666,0.018182,0.002630,0.006897,0.004167,0.001287,0.004329,0.018519,0.001723,0.001848,0.004739,0.001423,0.004785,0.021978,0.002950,0.002415,0.002601,0.005747,0.022727,0.028986
FW,0.006211,0.022727,0.000173,0.023256,0.006579,0.000147,0.016393,0.000883,0.000199,0.000405,0.002532,0.000591,0.000571,0.000580,0.02439,0.020408,0.001590,0.000483,0.023810,0.000210,0.007299,0.000133,0.018182,0.000526,0.013793,0.004167,0.001287,0.004329,0.018519,0.000861,0.000924,0.004739,0.000356,0.004785,0.010989,0.002950,0.001208,0.000217,0.005747,0.022727,0.014493
DT,0.012422,0.045455,0.002418,0.023256,0.072368,0.122470,0.016393,0.007061,0.306169,0.004860,0.010127,0.002957,0.002283,0.045244,0.02439,0.020408,0.003180,0.000483,0.023810,0.107511,0.021898,0.009317,0.018182,0.032614,0.013793,0.008333,0.014157,0.103896,0.018519,0.056848,0.007394,0.028436,0.073639,0.004785,0.010989,0.120944,0.018116,0.292868,0.327586,0.022727,0.246377
UH,0.006211,0.022727,0.000173,0.023256,0.006579,0.000147,0.016393,0.000883,0.000100,0.000405,0.002532,0.000591,0.000571,0.000580,0.02439,0.020408,0.001590,0.000483,0.023810,0.000210,0.007299,0.000266,0.018182,0.000526,0.006897,0.004167,0.001287,0.004329,0.018519,0.000861,0.000924,0.004739,0.000356,0.004785,0.010989,0.002950,0.000604,0.000217,0.005747,0.022727,0.014493
RBR,0.012422,0.022727,0.001036,0.023256,0.006579,0.000587,0.016393,0.001765,0.000100,0.000810,0.002532,0.001774,0.001712,0.002900,0.02439,0.020408,0.001590,0.001450,0.023810,0.000210,0.014599,0.003328,0.018182,0.007891,0.006897,0.004167,0.002574,0.004329,0.018519,0.002584,0.001848,0.004739,0.000356,0.004785,0.010989,0.002950,0.001208,0.009538,0.005747,0.022727,0.014493
NNP,0.086957,0.022727,0.038694,0.023256,0.006579,0.463186,0.016393,0.024713,0.064677,0.225192,0.134177,0.209344,0.031963,0.023782,0.02439,0.020408,0.004769,0.005317,0.047619,0.046918,0.240876,0.053108,0.072727,0.050500,0.234483,0.083333,0.108108,0.562771,0.018519,0.028424,0.040665,0.175355,0.081110,0.009569,0.010989,0.017699,0.191425,0.038370,0.051724,0.022727,0.043478
PDT,0.006211,0.022727,0.003628,0.023256,0.006579,0.000147,0.016393,0.000883,0.000100,0.000405,0.002532,0.000591,0.000571,0.000580,0.02439,0.020408,0.006359,0.000483,0.023810,0.000210,0.007299,0.000133,0.018182,0.000526,0.006897,0.004167,0.001287,0.004329,0.018519,0.000861,0.000924,0.004739,0.000356,0.004785,0.010989,0.002950,0.000604,0.000217,0.005747,0.022727,0.014493
PRP,0.006211,0.022727,0.005182,0.023256,0.013158,0.000880,0.016393,0.006178,0.000995,0.144593,0.002532,0.011236,0.017123,0.005800,0.02439,0.040816,0.001590,0.006283,0.023810,0.000631,0.029197,0.007321,0.018182,0.054708,0.013793,0.020833,0.209781,0.004329,0.018519,0.018949,0.199630,0.009479,0.000356,0.033493,0.010989,0.017699,0.158213,0.005636,0.005747,0.022727,0.014493
NN,0.204969,0.068182,0.044222,0.023256,0.065789,0.037841,0.016393,0.114740,0.145672,0.247064,0.369620,0.306328,0.224315,0.070186,0.02439,0.122449,0.015898,0.012566,0.023810,0.195035,0.145985,0.346732,0.054545,0.124145,0.179310,0.275000,0.234234,0.008658,0.018519,0.124892,0.044362,0.151659,0.020989,0.023923,0.109890,0.070796,0.333937,0.039237,0.051724,0.022727,0.028986
VBD,0.043478,0.022727,0.072551,0.046512,0.059211,0.023467,0.032787,0.135922,0.009751,0.005265,0.002532,0.006505,0.083333,0.153712,0.02439,0.020408,0.092210,0.014983,0.023810,0.012834,0.007299,0.040729,0.018182,0.073645,0.006897,0.004167,0.006435,0.004329,0.018519,0.054264,0.001848,0.004739,0.058698,0.181818,0.054945,0.064897,0.002415,0.029699,0.011494,0.022727,0.014493


Applying Viterbi Algorithm

In [177]:
def Viterbi(words):
    state = []
    T = list(tagss) # Set of tags available
    l1 = 0.2       # Lambda Values
    l2 = 0.3
    l3 = 0.5
    for key, word in enumerate(words):
        p = [] 
        
        for tag in T:
            ind = T.index(tag)
            if key == 0:
                ind = T.index(tag)     #  Start Transition Probability
                
                transition_p = prior_prob[ind]
            elif key==1:
                transition_p = prior_prob[ind]*dfb.loc[state[-1],tag]  # Transition probability in case of second word
            else:
                tup = (state[-2],state[-1])               # Looking at previous two tags in case of 3rd or more word
                if tup in dft.index:
                  transition_p = l1*p_p[ind]+ l2*dfb.loc[state[-1],tag]+l3*dft.loc[[tup],tag][0]
                else:
                  transition_p = l1*p_p[ind]+ l2*dfb.loc[state[-1],tag]+l3*1/(dft.loc[:,tag].min())
                 
            emission_p = df.loc[tag,word]                 # Emmision Probability
            state_probability = emission_p * transition_p    # Probability of tag for given word
            p.append(state_probability)   
        pmax = max(p)                               # Taking tag with maximum probability among all other tags
        state_max = T[p.index(pmax)] 
        state.append(state_max)                     # Storing the max probability tag for given word in list
    return list(zip(words, state))

In [178]:
def pre_proc(s):           # Function to preprocess string
  s =re.sub(r'[^\w\s]', '', s).lower()    # Removing Punctuation
  s = s.split(" ")
  return s

Applying Viterbi Algorithm for each sentence of test set and Storing the return tuple in List

In [179]:
tsz = len(test_set)
count=0
total=0
reslt_L = []
match_L = []
for i in range(tsz):
  match = []
  rt = test_set.iloc[i]
  ttag = rt[1]
  tword = pre_proc(str(rt[0]))
  sn = len(tword)
  for j in range(sn):
    tp = (tword[j],ttag[j])
    match.append(tp)
  match_L.append(match)     # Storing each sentence word-tag tuple for test set in list
  reslt = Viterbi(tword)
  reslt_L.append(reslt)     # Storing each sentence word-tag tuple for return output set in list
  check = [i for i, j in zip(reslt, match) if i == j]
  count = count + len(check)
  total = total + len(reslt)

accuracy = count/total*100     
print("Overall accuracy: "+str(accuracy))        # Printing Overall Accuracy

Overall accuracy: 82.87272504442674


Calculating Class Wise Accuracy

In [180]:
lt = len(match_L)
ltg = len(tagss)
accur = [0]*ltg
t_count=0
tg_c=0
for i in range(ltg):
  tag = list(tagss)[i]
  ind = list(tagss).index(tag)
  for j in range(lt):
    r_list = reslt_L[j]
    m_list = match_L[j]
    inL= len(r_list)
    for k in range(inL):
      if(m_list[k][1]==tag):
        t_count+=1
        if(r_list[k][1]==tag):
          tg_c+=1
  if(t_count!=0):
    accur[ind] = tg_c/t_count*100
  t_count=0
  tg_c=0

    
print("Class Wise Accuracy")
for i in range(ltg):
  print(list(tagss)[i] + ": " +str(accur[i]))

  

Class Wise Accuracy
WRB: 89.74358974358975
FW: 0.0
DT: 98.97652016857315
UH: 0.0
RBR: 4.3478260869565215
NNP: 73.33664349553129
PDT: 0.0
PRP: 91.98813056379822
NN: 87.65625
VBD: 79.11908646003263
WDT: 50.56179775280899
CC: 96.16252821670429
TO: 99.78021978021978
VBN: 66.74418604651163
SYM: 0.0
'': 0.0
PRP$: 96.0
VB: 85.29411764705883
,: 0
NNS: 76.64418212478921
-LRB-: 84.21052631578947
IN: 99.07928388746802
WP$: 0
RB: 69.27966101694916
-RRB-: 100.0
:: 72.09302325581395
MD: 93.54838709677419
NNPS: 31.11111111111111
#: 66.66666666666666
VBG: 42.10526315789473
VBP: 66.80851063829788
WP: 92.72727272727272
CD: 76.10738255033557
RP: 31.25
EX: 50.0
JJR: 70.83333333333334
VBZ: 83.41708542713567
JJ: 67.63925729442971
JJS: 70.45454545454545
LS: 0.0
RBS: 20.0


# HMM Model Using Only 4 Tags

Classfying all Tags into 4 tags (Collapsing)

In [181]:
noun = ['NN','NNS','NNP','NNPS']
verb = ['VB','VBD','VBG','VBN','VBP','VBZ']
adverb = ['RB','RBR','RBS']
rlen = len(raw_data)
for i in range(rlen):
  rd = raw_data.iloc[i]
  rl = len(rd[1])
  for j in range(rl):
    if rd[1][j] in noun:
      rd[1][j] = 'N'
    elif rd[1][j] in verb:
      rd[1][j] = 'V'
    elif rd[1][j] in adverb:
      rd[1][j] = 'A'
    else:
      rd[1][j] = 'O'
raw_data
#NN, NN, VBZ, IN, NNS


,0,1
0,"[pierre, vinken, 61, years, old, will, join, t...","[N, N, O, N, O, O, V, O, N, O, O, O, N, N, O]"
1,"[mr, vinken, is, chairman, of, elsevier, nv, t...","[N, N, V, N, O, N, N, O, N, V, N]"
2,"[rudolph, agnew, 55, years, old, and, former, ...","[N, N, O, N, O, O, O, N, O, N, N, N, N, V, V, ..."
3,"[a, form, of, asbestos, once, used, to, make, ...","[O, N, O, N, A, V, O, V, N, N, N, V, V, O, O, ..."
4,"[the, asbestos, fiber, crocidolite, is, unusua...","[O, N, N, N, V, A, O, O, O, V, O, N, O, A, O, ..."
...,...,...
3909,"[they, also, said, that, more, than, a, dozen,...","[O, A, V, O, O, O, O, N, N, V, V, O, O, N, N, ..."
3910,"[sen, kennedy, said, in, a, separate, statemen...","[N, N, V, O, O, O, N, O, O, V, N, O, V, O, N, ..."
3911,"[trinity, industries, inc, said, it, reached, ...","[N, N, N, V, O, V, O, O, N, O, V, O, N, N, O, ..."
3912,"[terms, werent, disclosed]","[N, V, V]"


Initialising the Emmision matrix with all 1 in row, columns

In [182]:
sz = len(raw_data)
df4 = pd.DataFrame()
tag_li = []
word_li = []

for i in range(sz):
  rd = raw_data.iloc[i]
  st = str(rd[0])
  lw =re.sub(r'[^\w\s]', '', st).lower()
  lw = lw.split(" ")
  raw_data.iloc[i][0]= lw
  
  tag_li.append(rd[1])
  word_li.append(lw)
tags = []
words = []
for lst in tag_li:
  for tag in lst:
    tags.append(tag)
tagss = set(tags)
for wrd in word_li:
  for word in wrd:
    words.append(word)
wordss = set(words)
print(len(words)) 
df4 = pd.DataFrame(index = tagss, columns = wordss)
df4= df4.fillna(1)
df4

81147


,,openly,tanked,cigna,patent,polystyrene,microprocessor,workable,published,file,lines,mossman,occurs,645000,ties,helps,reddington,needy,hotel,01,original,witness,13101,increasing,newspaper,152000,enraged,alleviate,chunks,riese,criticism,hammerton,bankroll,hero,flourish,stupid,attention,accounts,treasury,discontinue,...,eaton,after,truthful,ticket,facilitate,grange,1980,believed,understatement,incorporated,reporters,impending,ought,386,available,drink,98,minn,ten,released,droughts,rapport,waste,familyplanning,pressman,strapped,industry,collective,inaccurately,chassis,sewer,extensive,guffey,313,receivers,subcommittees,switched,passenger,routine,vision
A,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
O,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
V,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
N,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


Dividing the dataset into train test and test set in 80:20 Ration

In [183]:
train_set,test_set =train_test_split(raw_data,train_size=0.80,test_size=0.20,random_state = 101)
print(train_set)

                                                      0                                                  1
1998          [drink, carrier, competes, with, cartons]                                    [N, N, V, O, N]
1455  [at, last, count, candela, had, sold, 4, milli...         [O, O, N, N, V, V, O, O, O, O, O, N, O, N]
218   [mrs, hills, lauded, south, korea, for, creati...  [N, N, V, N, N, O, V, O, O, N, N, O, O, N, N, ...
2161  [section, 501, of, the, united, states, inform...  [N, O, O, O, N, N, N, O, N, N, N, O, O, V, N, ...
7     [a, lorillard, spokewoman, said, this, is, an,...                        [O, N, N, V, O, V, O, O, N]
...                                                 ...                                                ...
1599  [the, programtrading, issue, is, heating, up, ...  [O, O, N, V, V, O, O, N, N, O, O, V, O, N, N, ...
3910  [sen, kennedy, said, in, a, separate, statemen...  [N, N, V, O, O, O, N, O, O, V, N, O, V, O, N, ...
1361  [economic, news, had, little, e

In [184]:
# creating List of all words and tags
tagL = []
wordL = []
nn = len(train_set)
for i in range(nn):
  rd = train_set.iloc[i]
  st = str(rd[0])
  lw =re.sub(r'[^\w\s]', '', st).lower()
  lw = lw.split(" ")
  train_set.iloc[i][0]= lw
  
  tagL.append(rd[1])
  wordL.append(lw)
ln = len(tagL)

In [185]:
# Storing frequency of all word for given tag in emmision Matrix
for i in range(len(train_set)):
  rd = train_set.iloc[i]
  n = len(rd[0])
  
  tg = rd[1]
  wd = rd[0]
  for j in range(n):
    df4.loc[tg[j],wd[j]]+=1
df4


,,openly,tanked,cigna,patent,polystyrene,microprocessor,workable,published,file,lines,mossman,occurs,645000,ties,helps,reddington,needy,hotel,01,original,witness,13101,increasing,newspaper,152000,enraged,alleviate,chunks,riese,criticism,hammerton,bankroll,hero,flourish,stupid,attention,accounts,treasury,discontinue,...,eaton,after,truthful,ticket,facilitate,grange,1980,believed,understatement,incorporated,reporters,impending,ought,386,available,drink,98,minn,ten,released,droughts,rapport,waste,familyplanning,pressman,strapped,industry,collective,inaccurately,chassis,sewer,extensive,guffey,313,receivers,subcommittees,switched,passenger,routine,vision
A,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1
O,292,1,1,1,1,1,1,2,1,1,1,1,1,3,1,1,1,2,1,4,5,1,2,1,1,2,1,1,1,1,1,1,1,1,1,5,1,1,1,1,...,1,71,2,1,1,1,2,1,1,1,1,2,3,3,15,1,5,1,1,1,1,1,1,3,1,1,1,2,1,1,1,2,1,2,1,1,1,1,2,1
V,1,1,2,1,1,1,1,1,2,6,1,1,3,1,1,2,1,1,1,1,1,1,1,14,1,1,1,2,1,1,1,1,2,1,2,1,1,2,1,2,...,1,1,1,1,2,1,1,7,1,2,1,1,1,1,1,3,1,1,1,5,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,3,1,1,1
N,1,1,1,3,8,2,2,1,1,1,6,3,1,1,6,1,2,1,5,1,1,2,1,1,9,1,1,1,1,4,3,1,1,3,1,1,6,7,28,1,...,2,1,1,2,1,2,1,1,2,1,5,1,1,1,1,3,1,1,1,1,3,2,4,1,2,1,34,1,1,2,2,1,3,1,1,2,1,3,2,2


Calculating prior probability of each tag

In [186]:
nt = len(df4.iloc[0])
for i in range(nt):
  s = df4.iloc[:,i].sum()
  df4.iloc[:,i]/=s
df4

,,openly,tanked,cigna,patent,polystyrene,microprocessor,workable,published,file,lines,mossman,occurs,645000,ties,helps,reddington,needy,hotel,01,original,witness,13101,increasing,newspaper,152000,enraged,alleviate,chunks,riese,criticism,hammerton,bankroll,hero,flourish,stupid,attention,accounts,treasury,discontinue,...,eaton,after,truthful,ticket,facilitate,grange,1980,believed,understatement,incorporated,reporters,impending,ought,386,available,drink,98,minn,ten,released,droughts,rapport,waste,familyplanning,pressman,strapped,industry,collective,inaccurately,chassis,sewer,extensive,guffey,313,receivers,subcommittees,switched,passenger,routine,vision
A,0.003390,0.500000,0.2,0.166667,0.090909,0.2,0.2,0.2,0.2,0.111111,0.111111,0.166667,0.166667,0.166667,0.111111,0.2,0.2,0.2,0.125,0.142857,0.125,0.2,0.2,0.058824,0.083333,0.2,0.25,0.2,0.25,0.142857,0.166667,0.25,0.2,0.166667,0.2,0.125,0.111111,0.090909,0.032258,0.2,...,0.2,0.039474,0.2,0.2,0.2,0.2,0.2,0.1,0.2,0.2,0.125,0.2,0.166667,0.166667,0.055556,0.125,0.125,0.25,0.25,0.125,0.166667,0.2,0.142857,0.166667,0.2,0.2,0.027027,0.2,0.4,0.2,0.2,0.2,0.166667,0.2,0.25,0.2,0.166667,0.166667,0.166667,0.2
O,0.989831,0.166667,0.2,0.166667,0.090909,0.2,0.2,0.4,0.2,0.111111,0.111111,0.166667,0.166667,0.500000,0.111111,0.2,0.2,0.4,0.125,0.571429,0.625,0.2,0.4,0.058824,0.083333,0.4,0.25,0.2,0.25,0.142857,0.166667,0.25,0.2,0.166667,0.2,0.625,0.111111,0.090909,0.032258,0.2,...,0.2,0.934211,0.4,0.2,0.2,0.2,0.4,0.1,0.2,0.2,0.125,0.4,0.500000,0.500000,0.833333,0.125,0.625,0.25,0.25,0.125,0.166667,0.2,0.142857,0.500000,0.2,0.2,0.027027,0.4,0.2,0.2,0.2,0.4,0.166667,0.4,0.25,0.2,0.166667,0.166667,0.333333,0.2
V,0.003390,0.166667,0.4,0.166667,0.090909,0.2,0.2,0.2,0.4,0.666667,0.111111,0.166667,0.500000,0.166667,0.111111,0.4,0.2,0.2,0.125,0.142857,0.125,0.2,0.2,0.823529,0.083333,0.2,0.25,0.4,0.25,0.142857,0.166667,0.25,0.4,0.166667,0.4,0.125,0.111111,0.181818,0.032258,0.4,...,0.2,0.013158,0.2,0.2,0.4,0.2,0.2,0.7,0.2,0.4,0.125,0.2,0.166667,0.166667,0.055556,0.375,0.125,0.25,0.25,0.625,0.166667,0.2,0.142857,0.166667,0.2,0.4,0.027027,0.2,0.2,0.2,0.2,0.2,0.166667,0.2,0.25,0.2,0.500000,0.166667,0.166667,0.2
N,0.003390,0.166667,0.2,0.500000,0.727273,0.4,0.4,0.2,0.2,0.111111,0.666667,0.500000,0.166667,0.166667,0.666667,0.2,0.4,0.2,0.625,0.142857,0.125,0.4,0.2,0.058824,0.750000,0.2,0.25,0.2,0.25,0.571429,0.500000,0.25,0.2,0.500000,0.2,0.125,0.666667,0.636364,0.903226,0.2,...,0.4,0.013158,0.2,0.4,0.2,0.4,0.2,0.1,0.4,0.2,0.625,0.2,0.166667,0.166667,0.055556,0.375,0.125,0.25,0.25,0.125,0.500000,0.4,0.571429,0.166667,0.4,0.2,0.918919,0.2,0.2,0.4,0.4,0.2,0.500000,0.2,0.25,0.4,0.166667,0.500000,0.333333,0.4


Calculation of emmision Probability and storing it to emmision table

In [187]:
p_p=[0]*(len(tagss))
tagL = len(tags)
for i in tagss:
  count=0
  for j in tags:
    if(i==j):
      count+=1
    ind  = list(tagss).index(i)
    p_p[ind] = count/tagL
p_p


[0.0328786030290707,
 0.4629253083909448,
 0.15394284446744796,
 0.3502532441125365]

Initialisng Trigram transition probability Dataframe with all 1

In [188]:
BitagL = []
nn = len(raw_data)

for i in tagss:
  for j in tagss:
    triT = (i,j)
    BitagL.append(triT)
dft4 = pd.DataFrame(index = BitagL,columns=tagss)
dft4 = dft4.fillna(1)
dft4

,A,O,V,N
"(A, A)",1,1,1,1
"(A, O)",1,1,1,1
"(A, V)",1,1,1,1
"(A, N)",1,1,1,1
"(O, A)",1,1,1,1
"(O, O)",1,1,1,1
"(O, V)",1,1,1,1
"(O, N)",1,1,1,1
"(V, A)",1,1,1,1
"(V, O)",1,1,1,1


In [189]:
# storing frequency of all pair of tags suceeded by given tag
nn = len(train_set)
for i in range(nn):
  rd = train_set.iloc[i]
  tset = rd[1]
  for j in range(2,len(tset)):
    lt = (tset[j-2],tset[j-1])
    cur = tset[j]
    dft4.loc[[lt],cur]+=1
dft4


,A,O,V,N
"(A, A)",12,99,50,5
"(A, O)",33,537,136,336
"(A, V)",36,473,80,94
"(A, N)",2,37,25,41
"(O, A)",48,311,253,23
"(O, O)",199,4238,1091,6164
"(O, V)",226,2351,580,742
"(O, N)",328,5668,1868,3723
"(V, A)",59,376,166,18
"(V, O)",114,2677,795,2336


In [190]:
# Setting probability value for trigram sequence in trigram transition probability matrix
nt = len(dft4.iloc[0])
for i in range(nt):
  s = dft4.iloc[:,i].sum()
  dft4.iloc[:,i]/=s
dft4

,A,O,V,N
"(A, A)",0.006329,0.003649,0.005319,0.000247
"(A, O)",0.017405,0.019794,0.014468,0.016622
"(A, V)",0.018987,0.017435,0.008511,0.004650
"(A, N)",0.001055,0.001364,0.002660,0.002028
"(O, A)",0.025316,0.011463,0.026915,0.001138
"(O, O)",0.104958,0.156211,0.116064,0.304937
"(O, V)",0.119198,0.086657,0.061702,0.036707
"(O, N)",0.172996,0.208920,0.198723,0.184179
"(V, A)",0.031118,0.013859,0.017660,0.000890
"(V, O)",0.060127,0.098673,0.084574,0.115563


Calculating Start Probability of each tag

In [191]:

tagInd = list(tagss)
s = len(tagss)
prior_prob4 = [1]*s
for i in range(nn):
  rd = train_set.iloc[i]
  ini_tag = rd[1][0]
  ind = tagInd.index(ini_tag)
  prior_prob4[ind]+=1


In [192]:
sm = sum(prior_prob4)
for i in range(len(prior_prob4)):
  prior_prob4[i] = prior_prob4[i]/sm
print(prior_prob4)

[0.05422647527910686, 0.6210526315789474, 0.028070175438596492, 0.2966507177033493]


Calculating Bigram Transition Probability Dataframe

In [193]:
# Initialising Bigram Dataframe
dfb4 = pd.DataFrame(index = tagss,columns =tagss)
dfb4 = dfb4.fillna(1)

In [194]:
# Storing frequency of each pair of sequence of tag
nn = len(train_set)
for i in range(nn):
  rd = train_set.iloc[i]
  tset = rd[1]
  for j in range(1,len(tset)):
    dfb4.loc[tset[j-1],tset[j]]+=1
dfb4

,A,O,V,N
A,184,1073,703,105
O,650,11964,4016,12879
V,642,6031,1477,1487
N,527,9031,3723,7221


In [196]:
# Using frequency to Calculate Probability of Bigram transition
nt = len(dfb4.iloc[0])
for i in range(nt):
  s = dfb4.iloc[:,i].sum()
  dfb4.iloc[:,i]/=s
dfb4

,A,O,V,N
A,0.091862,0.038186,0.070874,0.004840
O,0.324513,0.425780,0.404880,0.593721
V,0.320519,0.214634,0.148906,0.068551
N,0.263105,0.321399,0.375340,0.332888


Applying Viterbi Algorithm

In [197]:
def Viterbi4(words):
    state = []
    T = list(tagss)    # Set of tags available
    l1 = 0.2           # Lambda Values
    l2 = 0.3
    l3 = 0.5
    for key, word in enumerate(words):
        p = [] 
        
        for tag in T:
            ind = T.index(tag)
            if key == 0:
                ind = T.index(tag)          #  Start Transition Probability
                transition_p = prior_prob4[ind]
            elif key==1:
                transition_p = prior_prob4[ind]*dfb4.loc[state[-1],tag]            # Transition probability in case of second word
                
            else:
                tup = (state[-2],state[-1])                  # Looking at previous two tags in case of 3rd or more word
                if tup in dft4.index:
                  transition_p = l1*p_p[ind]+ l2*dfb4.loc[state[-1],tag]+l3*dft4.loc[[tup],tag][0]
                else:
                  transition_p = l1*p_p[ind]+ l2*dfb4.loc[state[-1],tag]+l3*1/(dft4.loc[:,tag].min())
                 
            emission_p = df4.loc[tag,word]                    # Emmision Probability
            state_probability = emission_p * transition_p      # Probability of tag for given word  
            p.append(state_probability)                               
        pmax = max(p)                   # Taking tag with maximum probability among all other tags
        state_max = T[p.index(pmax)] 
        state.append(state_max)            # Storing the max probability tag for given word in list
    return list(zip(words, state))

Applying Viterbi Algorithm for each sentence of test set and Storing the return tuple in List

In [198]:
tsz = len(test_set)
count=0
total=0
match_L = []
reslt_L = []

for i in range(tsz):
  match = []
  rt = test_set.iloc[i]
  ttag = rt[1]
  tword = pre_proc(str(rt[0]))
  sn = len(tword)
  for j in range(sn):
    tp = (tword[j],ttag[j])
    match.append(tp)
  match_L.append(match)           # Storing each sentence word-tag tuple for test set in list
  reslt = Viterbi4(tword)
  reslt_L.append(reslt)         # Storing each sentence word-tag tuple for return output set in list
  check = [i for i, j in zip(reslt, match) if i == j]
  count = count + len(check)
  total = total + len(reslt)

accuracy = count/total*100
print("Overall accuracy: "+str(accuracy))                      # Printing Overall Accuracy

Overall accuracy: 90.3793124578712


Calculating Class Wise Accuracy

In [199]:
lt = len(match_L)
ltg = len(tagss)
accur = [0]*ltg
t_count=0
tg_c=0
for i in range(ltg):
  tag = list(tagss)[i]
  ind = list(tagss).index(tag)
  for j in range(lt):
    r_list = reslt_L[j]
    m_list = match_L[j]
    inL= len(r_list)
    for k in range(inL):
      if(m_list[k][1]==tag):
        t_count+=1
        if(r_list[k][1]==tag):
          tg_c+=1
  if(t_count!=0):
    accur[ind] = tg_c/t_count*100
  t_count=0
  tg_c=0

    
print("Class Wise Accuracy")
for i in range(ltg):
  print(list(tagss)[i] + ": " +str(accur[i]))

  

Class Wise Accuracy
A: 69.39999999999999
O: 95.5077086656034
V: 79.27710843373494
N: 90.30146425495262
